# Project: Mobile Manipulator Planning
**Course:** Roboterprogrammierung (Winter 25/26)  
**Institution:** Hochschule Karlsruhe - University of Applied Sciences (HKA)  

## 🎯 Objective
This notebook implements and evaluates a **Collision Checker** for a planar mobile robot consisting of a movable base and a 2-DOF rotatory arm. The goal is to benchmark path planning algorithms (LazyPRM, VisibilityPRM, etc.) in various environments while handling self-collisions and obstacle avoidance.

## ⚙️ Key Features
* **Mobile Base:** Free-form shape definition (2D).
* **Manipulator:** Configurable arm segments (Length, Thickness, Joint Limits).
* **Collision Detection:** Custom implementation handling Base-Obstacle, Arm-Obstacle, and Self-Collision (Arm-Base).

## Imports & Autoreload


In [ ]:
# --- Auto-reload modules when source code changes ---
# This is crucial so you don't have to restart the kernel when editing src/ files
%load_ext autoreload
%autoreload 2

# --- Standard Imports ---
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx

# --- Add 'src' directory to path to import modules ---
# Assuming this notebook is in the 'notebooks/' folder and source is in 'src/'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)
# IMPORTANT: To ensure that IPLazyPRM finds its dependencies (IPPerfMonitor),
# we explicitly add the src/planners folder to the path.
planners_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src', 'planners'))
if planners_path not in sys.path:
    sys.path.append(planners_path)

print(f"Project Root added to path: {project_root}")

# --- Custom Imports ---
from IPLazyPRM import LazyPRM
from IPVisibilityPRM import VisPRM
# (Uncomment these once you have created the files in src/)
# from src.collision_checker import CollisionChecker
# from src.planners.lazy_prm import LazyPRM
# from src.planners.visibility_prm import VisibilityPRM

print(f"HKA Planners imported successfully.")

## Robot Configuration

In [ ]:
# --- Robot Configuration ---

# 1. Base Shape definition (Example: A simple rectangle centered at 0,0)
# Defined as a list of (x, y) vertices relative to the robot center
ROBOT_BASE_SHAPE = [
    (-2, -1), (2, -1), (2, 1), (-2, 1)
]

# 2. Arm Configuration
# Format: [Length, Thickness, [Min_Angle, Max_Angle]]
# Based on assignment example: [5.1, 1, [-3.14, 3.14]]
ROBOT_ARM_CONFIG = [
    # Segment 1
    [1.0, 0.5, [-np.pi, np.pi]], 
    # Segment 2
    [1.0, 0.5, [-np.pi, np.pi]]
]
ROBOT_ARM_OFFSET = (1.5, 1.5)

# 3. Start Configuration
# Base Position (x, y, theta) + Joint Angles (q1, q2)
START_CONFIG = [0, 0, 0, 0.5, -0.5]

print("Robot Configuration loaded.")

## Import & Initialisation

In [ ]:
# --- Import Collision Checker ---
from src.collision_checker import CollisionChecker

# Initialize the Collision Checker instance
# We use the constants defined in the Configuration cell above
cc = CollisionChecker(ROBOT_BASE_SHAPE, ROBOT_ARM_CONFIG, ROBOT_ARM_OFFSET)

print("CollisionChecker initialized successfully.")

## Environment Setup

In [ ]:
# --- Define Obstacles ---
# Obstacles are defined as lists of (x, y) vertices (Polygons)

obs_wall = [
    (5, -5), (6, -5), (6, 5), (5, 5)
]

obs_box = [
    (-5, -5), (-5, 2.5), (2.5, 2.5), (2.5, -5)
]

# Load obstacles into the checker
cc.set_obstacles([obs_box])

print(f"Environment set up with {len(cc.obstacles)} obstacles.")

## Interactive Visualization

In [ ]:
from ipywidgets import interact, FloatSlider
import matplotlib.pyplot as plt
import numpy as np # Wichtig: Numpy importieren

def interact_robot(x, y, theta, q1, q2, check_self):
    """
    Interactive function to control the robot via sliders.
    """
    # 0. Aufräumen: Alte Plots schließen, um Speicherüberlauf/Crash zu verhindern
    plt.close('all')
    
    # 1. Update Self-Collision Flag
    # Fehler abfangen, falls cc noch nicht existiert oder Methode fehlt
    try:
        cc.toggle_self_collision(check_self)
    except AttributeError:
        pass # Falls alte Version der Klasse geladen ist
    
    # 2. Build Configuration Array (WICHTIG: Als Numpy Array definieren!)
    config = np.array([x, y, theta, q1, q2], dtype=float)
    
    # 3. Check Collision
    try:
        # Nutzung von pointInCollision (kompatibel mit Planner-Interface)
        # oder Fallback auf is_in_collision
        if hasattr(cc, 'pointInCollision'):
            is_collision = cc.pointInCollision(config)
        else:
            is_collision = cc.is_in_collision(config)
            
    except Exception as e:
        print(f"Fehler bei Kollisionsberechnung: {e}")
        is_collision = True # Sicherheitshalber als Kollision werten

    # 4. Visualization
    try:
        fig, ax = plt.subplots(figsize=(10, 8))
        
        # Draw Robot and Obstacles
        cc.draw(config, ax)
        
        # Setup Plot Limits
        ax.set_xlim(-10, 10)
        ax.set_ylim(-10, 10)
        ax.grid(True, linestyle='--', alpha=0.5)
        ax.set_aspect('equal') # Wichtig für korrekte Proportionen
        
        # Title Status
        status_color = 'red' if is_collision else 'green'
        status_text = "COLLISION DETECTED!" if is_collision else "Path Clear"
        ax.set_title(f"Status: {status_text} (Self-Check: {check_self})", 
                     color=status_color, fontweight='bold', fontsize=14)
        
        plt.show()
    except Exception as e:
        print(f"Fehler beim Zeichnen: {e}")

# Create Sliders
interact(interact_robot, 
         x=FloatSlider(min=-8, max=8, step=0.1, value=0),
         y=FloatSlider(min=-8, max=8, step=0.1, value=0),
         theta=FloatSlider(min=-3.14, max=3.14, step=0.1, value=0, description='Base Theta'),
         q1=FloatSlider(min=-3.14, max=3.14, step=0.1, value=0.5, description='Joint 1'),
         q2=FloatSlider(min=-3.14, max=3.14, step=0.1, value=-0.5, description='Joint 2'),
         check_self=True  # Checkbox for self-collision
        );

## Planning

In [ ]:
# 1. Limits definieren
# Diese werden nun im CollisionChecker gespeichert und von IPPRMBase abgerufen
limit_x = (-10, 10)
limit_y = (-10, 10)
limit_theta = (-np.pi, np.pi)
limit_q1 = ROBOT_ARM_CONFIG[0][2] # [-3.14, 3.14] aus Config
limit_q2 = ROBOT_ARM_CONFIG[1][2]

sampling_limits = [limit_x, limit_y, limit_theta, limit_q1, limit_q2]

# Update Limits im Checker
cc.set_sampling_limits(sampling_limits)

# 2. Planer instanziieren
# LazyPRM(collChecker) -> ruft intern super().__init__(collChecker) auf
planner_lazy = LazyPRM(cc)

# 3. Planungseinstellungen (Dictionary)
config_lazy = {
    "initialRoadmapSize": 50,
    "updateRoadmapSize": 20,
    "kNearest": 5,
    "maxIterations": 50
}

# 4. Start und Ziel definieren
# WICHTIG: Die PlanerBase erwartet Listen von Listen/Arrays [[x,y...]]
start_conf = [[-7.5, -7.5, 0.0, 0.0, 0.0]] 
goal_conf  = [[4.0, 4.0, 1.57, 1.0, -1.0]]

# Prüfen ob Start/Ziel valide sind (nur zur Sicherheit für uns)
if cc.pointInCollision(start_conf[0]) or cc.pointInCollision(goal_conf[0]):
    print("WARNING: Start or destination conflicts! The planner will complain.")

print("Starting LazyPRM planning...")

try:
    # planPath gibt eine Liste von Knoten-IDs oder Positionen zurück (je nach Implementierung)
    # In deiner Version gibt er 'path' zurück, was nx.shortest_path(nodes) ist.
    # Da die Nodes im Graph Attribute 'pos' haben, müssen wir die Koordinaten extrahieren.
    path_node_ids = planner_lazy.planPath(start_conf, goal_conf, config_lazy)
    
    if path_node_ids and len(path_node_ids) > 0:
        print(f"Path found! Length: {len(path_node_ids)} nodes.")
        
        # Pfad rekonstruieren (Node IDs -> Koordinaten)
        # LazyPRM speichert Koordinaten im Attribut 'pos' des Graphen
        path_coordinates = []
        for node_id in path_node_ids:
            pos = planner_lazy.graph.nodes[node_id]['pos']
            path_coordinates.append(pos)
            
    else:
        print("No path found.")
        path_coordinates = []
        
except Exception as e:
    print(f"Planning failed with error: {e}")

## Visualization

In [ ]:
if len(path_coordinates) > 0:
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xlim(-10, 10)
    ax.set_ylim(-10, 10)
    
    # Draw Environment
    for obs in cc.obstacles:
        x, y = obs.exterior.xy
        ax.fill(x, y, fc='gray', alpha=0.5)

    # Draw Path Trace
    # Wir zeichnen jeden 2. oder 3. Schritt, damit es nicht zu voll wird
    for i, config in enumerate(path_coordinates):
        config_array = np.array(config) 
        geo = cc.get_robot_geometry(config_array)
        
        # Base (Ghost)
        bx, by = geo['base'].exterior.xy
        ax.plot(bx, by, color='blue', alpha=0.1)
        
        # Arm (Ghost)
        for seg in geo['arm_segments']:
            sx, sy = seg.exterior.xy
            ax.plot(sx, sy, color='orange', alpha=0.1)
            
    # Start (Grün) und Ziel (Rot) hervorheben
    cc.draw(path_coordinates[0], ax) # Start wird durch Standard-Farben überdeckt, evtl. anpassen
    cc.draw(path_coordinates[-1], ax)
    # Ziel manuell zeichnen für Highlight
    geo_goal = cc.get_robot_geometry(path_coordinates[-1])
    ax.plot(*geo_goal['base'].exterior.xy, color='green', linewidth=2, label='Goal')

    ax.set_title(f"LazyPRM Solution ({len(path_coordinates)} steps)")
    plt.show()